In [57]:
# TASK - 1
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.datasets.samples_generator import make_blobs
from sklearn.cluster import KMeans
import keras
from sklearn.utils.linear_assignment_ import linear_assignment
import util_mnist_reader
from sklearn import metrics
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
x_train, y_train = util_mnist_reader.load_mnist('data/fashion', kind='train')
x_test, y_test = util_mnist_reader.load_mnist('data/fashion', kind='t10k')
m=[]
for i in range (7,13):   
    kmeans=KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
        n_clusters=i, n_init=10, n_jobs=1, precompute_distances='auto',
        random_state=0, tol=0.0001, verbose=0)
    kmeans.fit(x_train)    
    m.append(kmeans.inertia_)
    print('inertia',m)
plt.figure(1)
plt.title('Elbow Graph')
plt.xlabel("Number of CLusters")
plt.ylabel("Inertia")
plt.plot(range(7,13),m,marker='o', markerfacecolor='blue', markersize=8)
o=[]
kmeans10=KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
        n_clusters=10, n_init=10, n_jobs=1, precompute_distances='auto',
        random_state=0, tol=0.0001, verbose=0)
kmeans10.fit(x_train)    
o.append(kmeans10.inertia_)
print('inertia',o)
kmeans10.fit(x_train)
p3=kmeans10.predict(x_test)
k10=kmeans10.inertia_
ConfusionMatrix=confusion_matrix(y_test,p3)  #CONFUSION MATRIX
from sklearn.utils.linear_assignment_ import linear_assignment
def _make_cost_m(ConfusionMatrix):
    s = np.max(ConfusionMatrix)
    return (- ConfusionMatrix + s)
ConfusionMatrix = confusion_matrix(y_test, p3)
indexes = linear_assignment(_make_cost_m(ConfusionMatrix))
js = [e[1] for e in sorted(indexes, key=lambda x: x[0])]
cm2 = ConfusionMatrix[:, js]
print('ConfusionMatrix')
print(cm2)
acc=np.trace(cm2) / np.sum(cm2)
print('accuracy',acc)

KeyboardInterrupt: 

In [ ]:
#TASK - 2
import keras
from matplotlib import pyplot as plt
import numpy as np
import gzip
%matplotlib inline
from keras.models import Model
from keras.optimizers import RMSprop
from keras.layers import Input,Dense,Flatten,Dropout,merge,Reshape,Conv2D,MaxPooling2D,UpSampling2D,Conv2DTranspose
from keras.layers.normalization import BatchNormalization
from keras.models import Model,Sequential
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adadelta, RMSprop,SGD,Adam
from keras import regularizers
from keras import backend as K
from keras.utils import to_categorical
def extract_data(filename, num_images):
    with gzip.open(filename) as bytestream:
        bytestream.read(16)
        buf = bytestream.read(28 * 28 * num_images)
        data = np.frombuffer(buf, dtype=np.uint8).astype(np.float32)
        data = data.reshape(num_images, 28,28)
        return data
train_data = extract_data('data/fashion/train-images-idx3-ubyte.gz', 60000)
test_data = extract_data('data/fashion/t10k-images-idx3-ubyte.gz', 10000)
def extract_labels(filename, num_images):
    with gzip.open(filename) as bytestream:
        bytestream.read(8)
        buf = bytestream.read(1 * num_images)
        labels = np.frombuffer(buf, dtype=np.uint8).astype(np.int64)
        return labels
train_labels = extract_labels('data/fashion/train-labels-idx1-ubyte.gz',60000)
test_labels = extract_labels('data/fashion/t10k-labels-idx1-ubyte.gz',10000)
# print('xtrainshape',train_data.shape)
# print('ytrainshape',test_data.shape)
# print('xtest shape',train_labels.shape)
# print('ytest',test_labels.shape)
label_dict = {
 0: 'A',
 1: 'B',
 2: 'C',
 3: 'D',
 4: 'E',
 5: 'F',
 6: 'G',
 7: 'H',
 8: 'I',
 9: 'J',
}
plt.figure(figsize=[5,5])
plt.subplot(121)
curr_img = np.reshape(train_data[10], (28,28))
curr_lbl = train_labels[10]
plt.imshow(curr_img, cmap='gray')
plt.title("(Label: " + str(label_dict[curr_lbl]) + ")")

# Display the first image in testing data
plt.subplot(122)
curr_img = np.reshape(test_data[10], (28,28))
curr_lbl = test_labels[10]
plt.imshow(curr_img, cmap='gray')
plt.title("(Label: " + str(label_dict[curr_lbl]) + ")")
# Text(0.5,1,'(Label: E)')
train_data = train_data.reshape(-1, 28,28, 1)
test_data = test_data.reshape(-1, 28,28, 1)
train_data.shape, test_data.shape
# print('xtrainshape',train_data.shape)
# print('ytrainshape',test_data.shape)
# print('xtest shape',train_labels.shape)
# print('ytest',test_labels.shape)

train_data.dtype, test_data.dtype
np.max(train_data), np.max(test_data)
train_data = train_data / np.max(train_data)
test_data = test_data / np.max(test_data)
np.max(train_data), np.max(test_data)
from sklearn.model_selection import train_test_split
train_X,valid_X,train_ground,valid_ground = train_test_split(train_data,
                                                             train_data,
                                                             test_size=0.2,
                                                             random_state=13)
batch_size = 64
epochs = 100
inChannel = 1
x, y = 28, 28
input_img = Input(shape = (x, y, inChannel))
num_classes = 10

def encoder(input_img):
    #encoder
    #input = 28 x 28 x 1 (wide and thin)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img) #28 x 28 x 32
    conv1 = BatchNormalization()(conv1)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    conv1 = BatchNormalization()(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1) #14 x 14 x 32
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1) #14 x 14 x 64
    conv2 = BatchNormalization()(conv2)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
    conv2 = BatchNormalization()(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2) #7 x 7 x 64
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2) #7 x 7 x 128 (small and thick)
    conv3 = BatchNormalization()(conv3)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
    conv3 = BatchNormalization()(conv3)
    conv4 = Conv2D(16, (3, 3), activation='relu', padding='same')(conv3) #7 x 7 x 256 (small and thick)
    conv4 = BatchNormalization()(conv4)
    conv4 = Conv2D(16, (3, 3), activation='relu', padding='same')(conv4)
    conv4 = BatchNormalization()(conv4)
    return conv4

def decoder(conv4):    
    #decoder
    conv5 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv4) #7 x 7 x 128
    conv5 = BatchNormalization()(conv5)
    conv5 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv5)
    conv5 = BatchNormalization()(conv5)
    conv6 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv5) #7 x 7 x 64
    conv6 = BatchNormalization()(conv6)
    conv6 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv6)
    conv6 = BatchNormalization()(conv6)
    up1 = UpSampling2D((2,2))(conv6) #14 x 14 x 64
    conv7 = Conv2D(32, (3, 3), activation='relu', padding='same')(up1) # 14 x 14 x 32
    conv7 = BatchNormalization()(conv7)
    conv7 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv7)
    conv7 = BatchNormalization()(conv7)
    up2 = UpSampling2D((2,2))(conv7) # 28 x 28 x 32
    decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(up2) # 28 x 28 x 1
    return decoded
autoencoder = Model(input_img, decoder(encoder(input_img)))
autoencoder.compile(loss='mean_squared_error', optimizer = RMSprop())
autoencoder.summary()
# print('train_x',train_X.shape)
# print('valid',valid_X.shape)
# print('train_x',train_X.shape)
# print('valid',valid_X.shape)
autoencoder_train = autoencoder.fit(train_X, train_ground, batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(valid_X, valid_ground))
loss = autoencoder_train.history['loss']
val_loss = autoencoder_train.history['val_loss']



#kmeans
from sklearn.cluster import MiniBatchKMeans
from sklearn.mixture import GaussianMixture
from sklearn import metrics
from sklearn.cluster import KMeans
encoder = Model(inputs=autoencoder.input, outputs=autoencoder.get_layer('conv2d_8').output)
kmeans = KMeans(n_clusters = 10, verbose=1,init= 'k-means++', n_init=30, random_state=50)
encoded = encoder.predict(test_data)
# print(encoded.shape)
encoder_pred = encoder.predict(test_data).reshape(test_data.shape[0], -1)
y_pred = kmeans.fit_predict(encoder_pred)

# print(y_pred.shape)

import matplotlib.pyplot as plt
plt.plot(autoencoder.history.history['loss'], color='#009358', marker='o')
plt.plot(autoencoder.history.history['val_loss'], color='red', marker='o')
plt.title("Training & validation loss")
plt.ylabel("Loss")
plt.xlabel("Epoch")
plt.legend(['Training loss', 'Validation loss'])
graph = plt.gcf()
plt.show()
graph.savefig('Task2_3_epoch_loss.png', dpi=300)


# print(test_labels.shape)
# print(y_pred.shape)
cm = metrics.confusion_matrix(test_labels, y_pred)

k_labels = y_pred  # Get cluster labels
k_labels_matched = np.empty_like(k_labels)

# For each cluster label...
for k in np.unique(k_labels):

    # ...find and assign the best-matching truth label
    match_nums = [np.sum((k_labels==k)*(test_labels==t)) for t in np.unique(test_labels)]
    k_labels_matched[k_labels==k] = np.unique(test_labels)[np.argmax(match_nums)]

cm1 = metrics.confusion_matrix(test_labels, k_labels_matched)

from sklearn.utils.linear_assignment_ import linear_assignment
def _make_cost_m(cm):
    s = np.max(cm)
    return (- cm + s)

indexes = linear_assignment(_make_cost_m(cm))
js = [e[1] for e in sorted(indexes, key=lambda x: x[0])]
cm2 = cm[:, js]

print ("Task 2 Confusion Matrix")
print(cm2)

print("(Task2) Accuracy of K-Means:",np.trace(cm2) / np.sum(cm2))



In [ ]:
#TASK - 3
import keras
from matplotlib import pyplot as plt
import numpy as np
import gzip
%matplotlib inline
from keras.models import Model
from keras.optimizers import RMSprop
from keras.layers import Input,Dense,Flatten,Dropout,merge,Reshape,Conv2D,MaxPooling2D,UpSampling2D,Conv2DTranspose
from keras.layers.normalization import BatchNormalization
from keras.models import Model,Sequential
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adadelta, RMSprop,SGD,Adam
from keras import regularizers
from keras import backend as K
from keras.utils import to_categorical
def extract_data(filename, num_images):
    with gzip.open(filename) as bytestream:
        bytestream.read(16)
        buf = bytestream.read(28 * 28 * num_images)
        data = np.frombuffer(buf, dtype=np.uint8).astype(np.float32)
        data = data.reshape(num_images, 28,28)
        return data
train_data = extract_data('data/fashion/train-images-idx3-ubyte.gz', 60000)
test_data = extract_data('data/fashion/t10k-images-idx3-ubyte.gz', 10000)
def extract_labels(filename, num_images):
    with gzip.open(filename) as bytestream:
        bytestream.read(8)
        buf = bytestream.read(1 * num_images)
        labels = np.frombuffer(buf, dtype=np.uint8).astype(np.int64)
        return labels
train_labels = extract_labels('data/fashion/train-labels-idx1-ubyte.gz',60000)
test_labels = extract_labels('data/fashion/t10k-labels-idx1-ubyte.gz',10000)

# print('xtrainshape',train_data.shape)
# print('ytrainshape',test_data.shape)
# print('xtest shape',train_labels.shape)
# print('ytest',test_labels.shape)
label_dict = {
 0: 'A',
 1: 'B',
 2: 'C',
 3: 'D',
 4: 'E',
 5: 'F',
 6: 'G',
 7: 'H',
 8: 'I',
 9: 'J',
}

plt.figure(figsize=[5,5])


plt.subplot(121)
curr_img = np.reshape(train_data[10], (28,28))
curr_lbl = train_labels[10]
plt.imshow(curr_img, cmap='gray')
plt.title("(Label: " + str(label_dict[curr_lbl]) + ")")

# Display the first image in testing data
plt.subplot(122)
curr_img = np.reshape(test_data[10], (28,28))
curr_lbl = test_labels[10]
plt.imshow(curr_img, cmap='gray')
plt.title("(Label: " + str(label_dict[curr_lbl]) + ")")
# Text(0.5,1,'(Label: E)')
train_data = train_data.reshape(-1, 28,28, 1)
test_data = test_data.reshape(-1, 28,28, 1)
train_data.shape, test_data.shape
# print('xtrainshape',train_data.shape)
# print('ytrainshape',test_data.shape)
# print('xtest shape',train_labels.shape)
# print('ytest',test_labels.shape)

train_data.dtype, test_data.dtype
np.max(train_data), np.max(test_data)
train_data = train_data / np.max(train_data)
test_data = test_data / np.max(test_data)
np.max(train_data), np.max(test_data)
from sklearn.model_selection import train_test_split
train_X,valid_X,train_ground,valid_ground = train_test_split(train_data,
                                                             train_data,
                                                             test_size=0.2,
                                                             random_state=13)
batch_size = 64
epochs = 100
inChannel = 1
x, y = 28, 28
input_img = Input(shape = (x, y, inChannel))
num_classes = 10

def encoder(input_img):
    #encoder
    #input = 28 x 28 x 1 (wide and thin)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img) #28 x 28 x 32
    conv1 = BatchNormalization()(conv1)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    conv1 = BatchNormalization()(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1) #14 x 14 x 32
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1) #14 x 14 x 64
    conv2 = BatchNormalization()(conv2)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
    conv2 = BatchNormalization()(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2) #7 x 7 x 64
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2) #7 x 7 x 128 (small and thick)
    conv3 = BatchNormalization()(conv3)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
    conv3 = BatchNormalization()(conv3)
    conv4 = Conv2D(16, (3, 3), activation='relu', padding='same')(conv3) #7 x 7 x 256 (small and thick)
    conv4 = BatchNormalization()(conv4)
    conv4 = Conv2D(16, (3, 3), activation='relu', padding='same')(conv4)
    conv4 = BatchNormalization()(conv4)
    return conv4

def decoder(conv4):    
    #decoder
    conv5 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv4) #7 x 7 x 128
    conv5 = BatchNormalization()(conv5)
    conv5 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv5)
    conv5 = BatchNormalization()(conv5)
    conv6 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv5) #7 x 7 x 64
    conv6 = BatchNormalization()(conv6)
    conv6 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv6)
    conv6 = BatchNormalization()(conv6)
    up1 = UpSampling2D((2,2))(conv6) #14 x 14 x 64
    conv7 = Conv2D(32, (3, 3), activation='relu', padding='same')(up1) # 14 x 14 x 32
    conv7 = BatchNormalization()(conv7)
    conv7 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv7)
    conv7 = BatchNormalization()(conv7)
    up2 = UpSampling2D((2,2))(conv7) # 28 x 28 x 32
    decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(up2) # 28 x 28 x 1
    return decoded
autoencoder = Model(input_img, decoder(encoder(input_img)))
autoencoder.compile(loss='mean_squared_error', optimizer = RMSprop())
autoencoder.summary()
# print('train_x',train_X.shape)
# print('valid',valid_X.shape)
# print('train_x',train_X.shape)
# print('valid',valid_X.shape)
autoencoder_train = autoencoder.fit(train_X, train_ground, batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(valid_X, valid_ground))
loss = autoencoder_train.history['loss']
val_loss = autoencoder_train.history['val_loss']
epochs = range(100)
plt.figure()
plt.plot(epochs, loss, 'bo-', label='Training loss', color='blue')
plt.plot(epochs, val_loss, 'b.-', label='Validation loss',color='green')
plt.title('Training and validation loss')
plt.legend()
plt.show()
autoencoder.save_weights('autoencoder.h5')

#gmm
import numpy as np
from sklearn.cluster import MiniBatchKMeans
from sklearn.mixture import GaussianMixture
from sklearn import metrics
from sklearn.cluster import KMeans
encoder = Model(inputs=autoencoder.input, outputs=autoencoder.get_layer('conv2d_8').output)
encoded = encoder.predict(test_data)
encoder_pred = encoder.predict(test_data).reshape(test_data.shape[0], -1)

gmm = GaussianMixture(n_components=10, covariance_type='spherical', tol=0.02, verbose=1, n_init=10, init_params='kmeans', random_state=35 )

y_pred_gmm = gmm.fit_predict(encoder_pred)

# print(y_pred_gmm.shape)

cm = metrics.confusion_matrix(test_labels, y_pred_gmm)

from sklearn.utils.linear_assignment_ import linear_assignment
def _make_cost_m(cm):
    s = np.max(cm)
    return (- cm + s)

indexes = linear_assignment(_make_cost_m(cm))
js = [e[1] for e in sorted(indexes, key=lambda x: x[0])]
cm2 = cm[:, js]

print ("Task 3 Confusion Matrix")
print(cm2)

print("(Task3) Accuracy of GMM:",np.trace(cm2) / np.sum(cm2))